In [22]:
%pip install composable

Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install toolz

Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install more-itertools

Note: you may need to restart the kernel to use updated packages.


In [1]:
from composable import pipeable
from composable.strict import map, filter

In [2]:
from more_itertools import with_iter

In [3]:
import records as rec

#### `records.Record`s allow access to keys as attributes, as well as the usual way

In [4]:
d1 = {'a': 5, 'b': 3}

In [5]:
(r1 := rec.Record(**d1))

record(a=5, b=3)

In [6]:
r1.a, r1['a']

(5, 5)

#### You can merge `Record` and/or `dict` with `|`

In [7]:
d2 = {'a': 1, 'c':5}

In [8]:
(r2 := rec.Record(**d2))

record(a=1, c=5)

In [9]:
r1 | r2, r2 | r1

(record(a=1, b=3, c=5), record(a=5, c=5, b=3))

In [10]:
r1 | d2, d2 | r1

(record(a=1, b=3, c=5), record(a=5, c=5, b=3))

## Using `records.create_record` to create a record from a value using a `dict` of functions.

In [11]:
identity = lambda x: x
add_one = lambda x: x+1
sqr = lambda x: x**2
cube = lambda x: x**3

In [12]:
(5
 >> rec.create(value = identity,
               plus_one = add_one,
               square = sqr,
               cube = cube,
               )
)

record(value=5, plus_one=6, square=25, cube=125)

In [13]:
(5
 >> rec.create(value = identity,
               plus_one = add_one,
               square = sqr,
               cube = cube,
               use_record_class=False)
)

{'value': 5, 'plus_one': 6, 'square': 25, 'cube': 125}

## Processing records with `map`, `apply` and `update`

* `records.map` applies a function to all values, leaving the keys unchanged.
* `records.apply` applies a function to the value of the specified key.
* `records.update` builds updated records by applying functions to the entire record, with the output mapped to the corresponding key.

In [14]:
(5
 >> rec.create(value = identity,
               use_record_class=False)
 >> rec.map(add_one)                               # map to all values
 >> rec.update(value = lambda r: r['value'] + 1,   # update the value at key 
               sqr = lambda r: r['value']**2,      # argument is whole record
              )
 >> rec.apply(value = add_one,                     # Apply a function only at the key's value
              sqr = str)                           # Argument is corresponding value
)                                                  # `dict` in ==> `dict` out

{'value': 8, 'sqr': '36'}

In [15]:
(5
 >> rec.create(value = identity,
               use_record_class=True)                  # Use the Record class (default)
 >> rec.map(add_one)
 >> rec.update(value = lambda r: r.value + 1,          # Records allow cleaner expressions 
               sqr = lambda r: r.value**2,             # by using attribute access to values
              )
 >> rec.apply(value = add_one,
              sqr = str)
 
)                                                      # `Record` in ==> `Record` out

record(value=8, sqr='36')

### `update` can process values simultaneously (default) or sequentially

When setting `sequential=True`, each subsequent function is passed the output of the previous function.  Note that this relies on `dict` maintaining insertion order, and thus requires Python 3.7+.

#### Non-sequential examples

In [16]:
(5
 >> rec.create(value = identity,
               use_record_class=True)
 >> rec.update(original = lambda r: r.value,  # save original
                value = lambda r: r.value + 1, # independently add one
                sqr = lambda r: r.value**2,    # independently square
                sequential=False
               )
)

record(value=6, original=5, sqr=25)

In [17]:
(5
 >> rec.create(value = identity,
               use_record_class=False)
 >> rec.update(original = lambda r: r['value'],  # save original
               value = lambda r: r['value'] + 1, # independently add one
               sqr = lambda r: r['value']**2,    # independently square
               sequential=False
              )
)

{'value': 6, 'original': 5, 'sqr': 25}

#### Sequential examples

In [18]:
(5
 >> rec.create(value = identity,
               use_record_class=False)
 >> rec.update(original = lambda r: r['value'],  # save original
               value = lambda r: r['value'] + 1, # And then add one
               sqr = lambda r: r['value']**2,    # And then sqr
               sequential=True
              )
)

{'value': 6, 'original': 5, 'sqr': 36}

In [19]:
(5
 >> rec.create(value = identity,
               use_record_class=True)
 >> rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
)

record(value=6, original=5, sqr=36)

### Why use `sequential=True`

The [let expression](https://en.wikipedia.org/wiki/Let_expression) is a common tool in expression-oriented functional programming languages that allows one to build a sequence of (local) bindings that are used in the body of the expression.  It is common that these bindings are processed sequentially, allowing newer expressions to use the variable defined in the previous expressions. This is particularly useful when breaking complicated computations into a sequence of more readable steps.

While it is difficult to implement a `let` expression on the user-side in Python, but we use `update` in an analogous way.   The main caveat is that we need to (A) house each "expression" in a `lambda` with the record as a parameter and (B) we need to access the elements of the record, either as attributes (`Record` only) or by scripting (both `dict` and `Record`s).  In fact, these expressions are the primary motivation for implementing the `Record` class.

In this case, you can think of the last keyword pair as the "body" or our let expression.

## Extracting values from a record with `get`

In [20]:
(5
 >> rec.create(value = identity,
               use_record_class=True)
 >> rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
 >> rec.get( 'sqr')
)

36

In [21]:
(5
 >> rec.create(value = identity,
               use_record_class=True)
 >> rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
 >> rec.get(['original', 'sqr'])
)

[5, 36]

In [22]:
(list(range(5))
 >> map(rec.create(value = identity,
                   use_record_class=True)
       )
 >> map(rec.update(original = lambda r: r.value,  # save original
                   value = lambda r: r.value + 1, # And then add one
                   sqr = lambda r: r.value**2,    # And then sqr
                   sequential=True
              )
       )
 >> map(rec.get('sqr'))
)

[1, 4, 9, 16, 25]

In [23]:
(list(range(5))
 >> map(rec.create(value = identity,
                   use_record_class=True)
       )
 >> map(rec.update(original = lambda r: r.value,  # save original
                   value = lambda r: r.value + 1, # And then add one
                   sqr = lambda r: r.value**2,    # And then sqr
                   sequential=True
                  )
       )
 >> map(rec.get(['original', 'sqr']))
)

[[0, 1], [1, 4], [2, 9], [3, 16], [4, 25]]

## Getting a `subset` of the key/value pairs

In [24]:
(5
 >> rec.create(value = identity,
               use_record_class=True)
 >> rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
 >> rec.subset(['original', 'sqr'])
)

record(original=5, sqr=36)

In [25]:
(list(range(5))
 >> map(rec.create(value = identity,
               use_record_class=True)
       )
 >> map(rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
       )
 >> map(rec.subset(['original', 'sqr']))
)

[record(original=0, sqr=1),
 record(original=1, sqr=4),
 record(original=2, sqr=9),
 record(original=3, sqr=16),
 record(original=4, sqr=25)]

In [26]:
(list(range(5))
 >> map(rec.create(value = identity,
               use_record_class=True)
       )
 >> map(rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
       )
 >> map(rec.subset(['original', 'sqr']))
 >> rec.zip()
)

{'original': [0, 1, 2, 3, 4], 'sqr': [1, 4, 9, 16, 25]}

In [27]:
(list(range(5))
 >> map(rec.create(value = identity,
               use_record_class=True)
       )
 >> map(rec.update(original = lambda r: r.value,  # save original
               value = lambda r: r.value + 1, # And then add one
               sqr = lambda r: r.value**2,    # And then sqr
               sequential=True
              )
       )
 >> rec.zip(keys = ['original', 'sqr'])
)

{'original': [0, 1, 2, 3, 4], 'sqr': [1, 4, 9, 16, 25]}